**$T_1.$** Soit $X_1$, ..., $X_n$ des variables aléatoires iid de loi $\mathcal{N}(\theta, 1)$, où $\theta$ est inconnu.
1.  (a) Montrer que $\widehat{\theta_n} = \frac{1}{n} \sum\limits_{k=1}^{n}X_k$ converge presque sûrement vers $\theta$.
    
    (b) Déterminer la loi de $\sqrt{n}(\widehat{\theta_n} - \theta)$.
1. Soit $\alpha$ $\in$ $]0, \;1[$ fixé et $\Phi$ la fonction répartition de la loi Gaussienne $\mathcal{N}(\theta, 1)$. Déterminer en fonction de $\alpha$, de $n$, de $\Phi^{-1}$ et de $\theta_0$ la valeur optimale de $K_\alpha$ pour que la relation $\mathbb{P}_{\theta = \theta_0}(T_K=1) \leq \alpha$ soit satisfaite avec une égalité.


//TODO T1 1.a, 1.b, 2

**$S_1.$** Créer un code python permettant:
1. de simuler $n$ variables gaussiennes indépendantes de loi $\mathcal{N}(\theta_0, 1)$,
1. de calculer pour $\alpha$ et $\theta_0$ fixés, la constante $K_\alpha$ et qui vous renvoie la valeur optimale de votre règle
de décision.
1. Pour $\theta_0 = 3$, $\alpha = 0.05$ et $n = 100$ répéter $N = 100$ fois l’expérience et compter le nombre fois où
vous avez décidé $H_0$. On appelle ce nombre $N_n$ quelle est sa loi?
1. Pour $n \in \{100,\; 1000\}$ et $N \in \{100, 1000, 5000\}$, Simuler $500$ valeurs de la variable aléatoire $N_n$ et
les représenter sous forme d’histogramme. Que remarquez-vous?

In [4]:
# Imports
import numpy as np

In [ ]:
#S1.1
def n_gauss_iid(theta0: float, n: int) -> np.ndarray:
    return np.random.normal(theta0, 1, n)